In [2]:
!pip install langgraph-supervisor langchain_openai python-dotenv langgraph langchain


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Un solo agente podría tener dificultades si necesita especializarse en múltiples dominios o gestionar muchas herramientas. Para solucionar esto, puede dividir su agente en agentes más pequeños e independientes y componerlos en un sistema multiagente.

En los sistemas multiagente, los agentes necesitan comunicarse entre sí. Lo hacen mediante transferencias, una primitiva que describe a qué agente transferir el control y la carga útil que se le debe enviar.

Dos de las arquitecturas multiagente más populares son:

- Supervisor: los agentes individuales son coordinados por un agente supervisor central. El supervisor controla todo el flujo de comunicación y la delegación de tareas, y decide qué agente invocar según el contexto actual y los requisitos de la tarea.

- Enjambre: los agentes se transfieren el control dinámicamente entre sí según sus especializaciones. El sistema recuerda qué agente estuvo activo por última vez, lo que garantiza que, en las interacciones posteriores, la conversación se reanude con ese agente.

### Supervisor
<div>
<img src="https://langchain-ai.github.io/langgraph/agents/assets/supervisor.png" width="500"/>
</div>

In [5]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
## from langgraph.prebuilt import create_react_agent # Deprecado, se movio a langchain
from langgraph_supervisor import create_supervisor
from dotenv import load_dotenv
load_dotenv()

def book_hotel(hotel_name: str):
    """Reservar un hotel"""
    return f"Successfully booked a stay at {hotel_name}."

def book_flight(from_airport: str, to_airport: str):
    """Agendar un vuelo"""
    return f"Successfully booked a flight from {from_airport} to {to_airport}."

flight_assistant = create_agent(
    model=ChatOllama(model="llama3.2:latest", temperature=0.2),
    tools=[book_flight],
    system_prompt="Eres un asistente que agenda vuelos",
    name="flight_assistant"
)

hotel_assistant = create_agent(
    model=ChatOllama(model="llama3.2:latest", temperature=0.2),
    tools=[book_hotel],
    system_prompt="Eres un asistente de reservas de hotel",
    name="hotel_assistant"
)

supervisor = create_supervisor(
    agents=[flight_assistant, hotel_assistant],
    #model=ChatOpenAI(model="gpt-4o-mini"),
    model=ChatOllama(model="llama3.2:latest", temperature=0.2),
    prompt=(
        "administras un asistente de reservas de hotel y"
        "un asistente que agenda vuelos. asignales trabajo para ellos."
    )
).compile()

for chunk in supervisor.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "agenda un vuelo de LAX a JFK y un hospedaje en Hotel Marriot para este fin de semana"
            }
        ]
    }
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='agenda un vuelo de LAX a JFK y un hospedaje en Hotel Marriot para este fin de semana', additional_kwargs={}, response_metadata={}, id='7e462506-65da-40c5-a7b5-29bff3b8ea4e'), AIMessage(content='{"name": "ask", "parameters": {"agent": "flight_assistant", "query": "agendar vuelo de LAX a JFK"}}; {"name": "ask", "parameters": {"agent": "hotel_assistant", "query": "reservar habitacion en Hotel Marriot para fin de semana"}}', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2026-02-13T00:40:22.8778758Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1240114200, 'load_duration': 165236400, 'prompt_eval_count': 222, 'prompt_eval_duration': 296911900, 'eval_count': 68, 'eval_duration': 718154400, 'logprobs': None, 'model_name': 'llama3.2:latest', 'model_provider': 'ollama'}, name='supervisor', id='lc_run--019c5471-0f83-76f2-82fe-322f2074227f-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'

# Swarm

![alt text](image.png)

Swarm se centra en que la coordinación y ejecución de agentes sea ligera, altamente controlable y fácil de probar.

Esto se logra mediante dos abstracciones primitivas: Agentes y transferencias. Un Agente comprende instrucciones y herramientas, y puede, en cualquier momento, transferir una conversación a otro Agente.

Estas primitivas son lo suficientemente potentes como para expresar una dinámica rica entre herramientas y redes de agentes, lo que permite crear soluciones escalables y prácticas, evitando una curva de aprendizaje pronunciada.

In [6]:
!pip install langgraph-swarm


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from langgraph.prebuilt import create_react_agent #deprecado
from langchain.agents import create_agent
from langgraph_swarm import create_swarm, create_handoff_tool

transfer_to_hotel_assistant = create_handoff_tool(
    agent_name="hotel_assistant",
    description="Transfiere al usuario con el asistente de reservas de hotel.",
)
transfer_to_flight_assistant = create_handoff_tool(
    agent_name="flight_assistant",
    description="Transfiere al usuario con el asistente de agendamiento de vuelos.",
)

flight_assistant = create_agent(
    model=ChatOllama(model="qwen3", temperature=0.2),
    tools=[book_flight, transfer_to_hotel_assistant],
    system_prompt="Eres un asistente que agenda vuelos de avion",
    name="flight_assistant"
)
hotel_assistant = create_agent(
    model=ChatOllama(model="qwen3", temperature=0.2),
    tools=[book_hotel, transfer_to_flight_assistant],
    system_prompt="Eres un asistente de reservas de hotel",
    name="hotel_assistant"
)

swarm = create_swarm(
    agents=[flight_assistant, hotel_assistant],
    default_active_agent="flight_assistant"
).compile()

for chunk in swarm.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "reserva un vuelo de BOS to JFK y un hospedaje en McKittrick Hotel"
            }
        ]
    }
):
    print(chunk)
    print("\n")

{'flight_assistant': {'messages': [HumanMessage(content='reserva un vuelo de BOS to JFK y un hospedaje en McKittrick Hotel', additional_kwargs={}, response_metadata={}, id='7a579dc5-a697-483b-8d5e-473da27c22d0'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3', 'created_at': '2026-02-13T00:54:54.5851344Z', 'done': True, 'done_reason': 'stop', 'total_duration': 17893057900, 'load_duration': 5423587800, 'prompt_eval_count': 223, 'prompt_eval_duration': 201388700, 'eval_count': 312, 'eval_duration': 12041925100, 'logprobs': None, 'model_name': 'qwen3', 'model_provider': 'ollama'}, name='flight_assistant', id='lc_run--019c547e-1b91-7000-907e-4215be90fe8a-0', tool_calls=[{'name': 'book_flight', 'args': {'from_airport': 'BOS', 'to_airport': 'JFK'}, 'id': '0f1a33e1-6f2c-4337-8553-2aabc511e2d4', 'type': 'tool_call'}, {'name': 'transfer_to_hotel_assistant', 'args': {}, 'id': '182d9a5a-d6c7-44d0-96da-aba574075967', 'type': 'tool_call'}], invalid_tool_calls=[], usa